In [1]:
#importing modules which is used in simple RAG Project.
#below classes we used so user can interact with LLM Models.
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint #HFE class we used to hit user query and take response from it.

#below classes we used for embedding Models
from langchain_openai import OpenAIEmbeddings  #close source model
from langchain_huggingface import HuggingFaceEmbeddings #open source model

import pdfplumber

#want to load document into workingspace using document loaders
from langchain_community.document_loaders import PDFPlumberLoader,TextLoader

#now splitting the document into chunks need splitter class
from langchain.text_splitter import RecursiveCharacterTextSplitter #this split the text document through herirachy way.

#need to store the chunks embedded document to vector store we r using Pinecone.
from pinecone import Pinecone,ServerlessSpec

#integrating pinecode with langchain.
from langchain_pinecone import PineconeVectorStore
from langsmith import traceable

#load the env files
from dotenv import load_dotenv

from pathlib import Path
from typing import Annotated,Optional,List,Literal,TypedDict
from dataclasses import dataclass
from loggers import logger
from Exception import CustomException
import os,sys

from langgraph.graph import StateGraph,START,END #using this class we can create Graph start or end of workflow

#if i want to add tool support to my workflow.
from langchain.tools import tool,Tool,StructuredTool

#if i want to add toolnode in my workflow 
#(toolnode means that node have list of tool here they will decide based on user query which tool need to execute)
from langgraph.prebuilt import ToolNode,tools_condition

#if i need to add memory or persistence to my workflow so that it can save the state value at every checkpoint
from langgraph.checkpoint.memory import InMemorySaver #it will save in state value to Ram memory.
from langgraph.checkpoint.sqlite import SqliteSaver

#tools_condition will  decide if tool message is present on AI response then they redirect to toolnode or end it workflow
from pydantic import BaseModel,Field,computed_field

#this class we used to change retriever object to become tool
from langchain_core.tools.retriever import create_retriever_tool

#fetching the RAG prompt from Hub.
from langchain import hub

import warnings as w
w.filterwarnings('ignore')

from langchain_core.messages import AIMessage,HumanMessage,AnyMessage,ToolMessage
load_dotenv()

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


True

## step:1) model objects

In [2]:
#groq model
model1 = ChatGroq(
    model="groq/compound-mini",
    temperature=0.1
)

#openai model
model2 = ChatOpenAI(
    model="gpt-3.5-turbo",temperature=0.1
)

#hugging face model.
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",  
    task="text-generation",  
    
)
model3 = ChatHuggingFace(llm=llm)


emb_model = OpenAIEmbeddings(model="text-embedding-3-small")

## Setting Up Pincone Database to store Embedding Vector into Index Folder

In [3]:
#Pinecone Basic Configuration.
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medical-book-index"

# 🔹 Ensure index exists
existing_indexes = [idx["name"] for idx in pc.list_indexes()]
existing_indexes

['medical-book-index']

In [4]:
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        tags={"environment": "RAGdevelopment"}
    )
    logger.info(f"🆕 Created Pinecone index: {index_name}")
    import time
    time.sleep(10)  # wait for index to be ready
else:
    logger.info(f"ℹ️ Index {index_name} already exists. Skipping creation.")

[2025-09-23 16:06:11,425]-INFO-13-ℹ️ Index medical-book-index already exists. Skipping creation.


In [5]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=emb_model)
vector_store

### step:2) changing vector store to become as retrievers

In [6]:
base_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2, "lambda_mult": 0.25} #lambda_mult will give more diversified output using MMR algorithm
)
base_retriever #retrievers is runnable so it means we can invoke easily.

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001D1A2816E60>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})

### step:3) Now changing vector store retriever to become tool

In [7]:
retriever_tool = create_retriever_tool(
    retriever=base_retriever,
    name="medical_doc_retriever",
    description=(
        "Use this tool to retrieve the most relevant information "
        "from the ingested PDFs. Best suited for answering "
        "questions that require factual context or reference to "
        "the uploaded documents."
    )
)
retriever_tool

Tool(name='medical_doc_retriever', description='Use this tool to retrieve the most relevant information from the ingested PDFs. Best suited for answering questions that require factual context or reference to the uploaded documents.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001D1C6715AB0>, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001D1A2816E60>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001D1C6715D80>, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langch

#### Note :- adding more tools to support

In [8]:
# Calculator tool
def calculator(first_num: float, second_num: float, operation: str) -> dict:
    try:
        if operation == "add": result = first_num + second_num
        elif operation == "sub": result = first_num - second_num
        elif operation == "mul": result = first_num * second_num
        elif operation == "div": result = first_num / second_num if second_num != 0 else "Division by zero"
        else: return {"error": f"Unsupported operation {operation}"}
        return {"first_num": first_num, "second_num": second_num, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}

calc_tool = Tool(
    name="Calculator",
    func=calculator,
    description="Perform basic arithmetic: add, sub, mul, div"
)

# Stock price tool
import requests
def get_stock_price(symbol: str) -> dict:
    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=1PPNPDOMK62HNKRO'
    return requests.get(url).json()

stock_tool = Tool(
    name="StockPrice",
    func=get_stock_price,
    description="Fetch latest stock price for a given symbol"
)

In [9]:
#now combining all tolls together
lst_tools = [retriever_tool,stock_tool,calc_tool]
lst_tools

[Tool(name='medical_doc_retriever', description='Use this tool to retrieve the most relevant information from the ingested PDFs. Best suited for answering questions that require factual context or reference to the uploaded documents.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001D1C6715AB0>, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001D1A2816E60>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001D1C6715D80>, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langc

In [10]:
#now binding the tool with LLM Model.
llm_with_tool  = model2.bind_tools(tools=lst_tools)
llm_with_tool  #tools is also runnable

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D1C697A800>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1C6979F90>, root_client=<openai.OpenAI object at 0x000001D1C697A2C0>, root_async_client=<openai.AsyncOpenAI object at 0x000001D1C7D57340>, temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'medical_doc_retriever', 'description': 'Use this tool to retrieve the most relevant information from the ingested PDFs. Best suited for answering questions that require factual context or reference to the uploaded documents.', 'parameters': {'properties': {'query': {'description': 'query to look up in retriever', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'StockPrice', 'description': 'Fetch latest stock price for a given symbol', 'p

## step:4) Defining State or Memory Schema that hold value throughout Workflow

In [11]:
from langgraph.graph.message import MessagesState #this message state is prebuilt class that store mesaage in lst 
from langgraph.graph.message import add_messages,BaseMessage

class StateSchema(TypedDict):
    #key schema defining i will store all the messages init as well as Query.
    messages : Annotated[list[BaseMessage],add_messages]

### step:5) Defineing the graph object and adding nodes and edges to graph so finally it ready the workflow

In [12]:
#creating an object of stategraph class
graph = StateGraph(state_schema=StateSchema)
graph

#### adding nodes and edges to my graph

In [ ]:
#creating user_query_or_respond function that perform action in node.
@traceable(
    name="user_query_or_respond_func",
    run_type="ParalleltoolsInvokeBasedUseryQuery",
    metadata={
        "module": "rag_agent",
        "owner": "Raees", 
        "description": "Decides whether to use a tool or respond directly",
        "version": "v1.0"
    },tags=["stateMsg","LLMBindWithtool","updateState"]
)
def user_query_or_respond(state:StateSchema) ->StateSchema:
    """Call the toool binded model to generate a response based on the current state(user query). Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    print("user_query_or_respond")
    #fetching the user query from state class.
    human_msg = state['messages'] #taking user input
    
    #now sending this user query that is LLM who is bind with tools.
    #it will parallely call all the tools together based on user query give suggestions which tool is suitable.
    ai_response = llm_with_tool.invoke(input=human_msg)
    
    #now updating the partial state message.
    return {
        'messages' : [ai_response]
    }

#### Tool nodes will have all tool support init

In [14]:
#if ai response coming from user_query_or_respond node have Toolmessage toh will redirect to toolnode.
#so defining logical that perform action in toolnode.
tools = ToolNode(tools=lst_tools)
tools

tools(tags=None, recurse=True, explode_args=False, func_accepts={'config': ('N/A', <class 'inspect._empty'>), 'store': ('store', None)}, tools_by_name={'medical_doc_retriever': Tool(name='medical_doc_retriever', description='Use this tool to retrieve the most relevant information from the ingested PDFs. Best suited for answering questions that require factual context or reference to the uploaded documents.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001D1C6715AB0>, retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001D1A2816E60>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=

```
## Very Important Meaning
👉 Grading documents ka matlab hai:
LLM (ya ek heuristic) ka use karke har retrieved document ko evaluate karna ki:
1)Kya yeh user query ke liye retrieve document relevant hai?
2)Kya isme user ko answer dene ke liye sahi context hai?
3)Agar multiple docs aaye to kaunsa sabse useful document hai?
```

In [15]:
#define GrageDocument schema using pydantic.
class GradeDocumentSchema(BaseModel):
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )
    


#### messages[0] → The original user question.
#### messages[1] → based on user question the assistant's placeholder,suggest which tool useful to solve the user question
#### messages[2] → The tool response (the document retrieved).

# Router Function based grade document suggestion it redirect

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

@traceable(
    name="grade_document_func",
    run_type="RouterFucnDecideDocRelatedQueryBasedOnRedirecting",
    metadata={
        "module": "rag_agent",
        "owner": "Raees", 
        "description": "Decidesto  whether a retrieved document is relevant to a given user question.",
        "version": "v1.0"
    },tags=["stateMsg","StructureBindingLLM","updateState"]
)

def grade_document(state: StateSchema) -> Literal['generate_answer','rewrite_question']:
    model2_structure = model2.with_structured_output(GradeDocumentSchema)
    
    question = state['messages'][0].content
    context = state['messages'][-1].content
    
    print("grade_document executed")
    # print(question)
    # print(context)
    
    prompt = PromptTemplate(
        template="""
        You are a relevance grader. 
        Your task is to decide whether a retrieved document is relevant to a given user question.

        Retrieved document:
        {context}

        User question:
        {question}

        Instructions:
        - Consider both exact keyword matches and semantic meaning.
        - If the document provides information that answers or is directly related to the question, grade it as 'yes'.
        - If the document does not provide relevant information, grade it as 'no'.
        Return the answer strictly in this format:
        """,
        input_variables=["context","question"],
    )
    
    # Render the prompt into a string
    prompt_text = prompt.format(context=context, question=question)
    
    # Pass as HumanMessage in a list
    response = model2_structure.invoke([HumanMessage(content=prompt_text,name='Human')])
    
    #print(response.binary_score)
    
    score = response.binary_score
    if score == "yes":
        print('relevant document')
        return "generate_answer"
    else:
        print('ir-relevant document')
        return "rewrite_question"


##### Generate response if document in relevant based on user query

In [ ]:
#creating function for generate node that perform action init.
@traceable(
    name="generate_answer_func",
    run_type="GeneratingAnswer",
    metadata={
        "module": "rag_agent",
        "owner": "Raees", 
        "description": "Generating Response based on User Query if document is relevant",
        "version": "v1.0"
    },tags=["stateMsg","GenerateAnswerLLM","updateState"]
)
def generate_answer(state:StateSchema)->StateSchema:
    print("geenerate answer executing")
    #fetching the user question and context from state class
    question = state["messages"][0].content
    context = state["messages"][-1].content
    
    
    #pulling the rag prompt from hub
    from langchain_core.prompts import PromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    rag_prompt = PromptTemplate(
        template="""
        
        You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        Use three sentences maximum and keep the answer concise.
        Question: {question} 
        Context: {context} 
        Answer:
        
        """,input_variables=['question','context']
    )
    
    # Render the prompt into a string
    prompt_text = rag_prompt.format(context=context, question=question)

    result = model2.invoke([HumanMessage(content=prompt_text,name='Human')])
    
    #now updating to state memory
    return {
        'messages': [result]
    }

#### if document ir-relevant based on user query than rewrite

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

@traceable(
    name="rewrite_question_func",
    run_type="RewritingQuestion",
    metadata={
        "module": "rag_agent",
        "owner": "Raees", 
        "description": "if document is irrelevant based on question rewriting",
        "version": "v1.0"
    },tags=["stateMsg","questionRewriting","updateState"]
)

# creating function for rewrite node
def rewrite_question(state: StateSchema) -> StateSchema:
    print("rewrite question executing")
    # fetching user question
    question = state['messages'][0].content

    # creating structured instruction prompt to rewrite
    hum_msg = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(
                "You are a helpful assistant that rewrites user questions to improve clarity and capture the true semantic intent."
            ),
            HumanMessagePromptTemplate.from_template(
                """Look at the input and try to reason about the underlying semantic intent or meaning.
                    Here is the initial question:
                    -------
                    {question}
                    -------
                    Formulate an improved question:"""
                                ),
                            ]
    )
    # Render the prompt into a string
    prompt_text = hum_msg.format(question=question)

    # invoke chain with question
    improved_question_response = model2.invoke([HumanMessage(content=prompt_text,name="Human")])

    #return the response to state.

    return {
        'messages':[improved_question_response]
    }


In [19]:
graph.add_node(node="understand_user_query_or_respond",action=user_query_or_respond)
graph.add_node(node = "vectorretriever", action=tools)
graph.add_node(node="rewrite_question",action=rewrite_question)
graph.add_node(node = "generate_answer",action=generate_answer)


In [20]:
#adding edges to graph.
graph.add_edge(START,"understand_user_query_or_respond")
graph.add_conditional_edges(
    "understand_user_query_or_respond"
    # Assess LLM decision (call `retriever_tool` tool or respond to the user)
    ,tools_condition,
    {
        "tools":"vectorretriever",
        END:END
    }
)

In [21]:
# Edges taken after the `action` node is called.
graph.add_conditional_edges(
    "vectorretriever",
    # router function will give 2 response in structure manner either doc is relevant or irrelevant.
    grade_document,{'generate_answer':"generate_answer",'rewrite_question':'rewrite_question'}
)

graph.add_edge('generate_answer',END)
graph.add_edge('rewrite_question','understand_user_query_or_respond')

### Testing the Agentic Rag

In [22]:
workflow = graph.compile()
messages=[HumanMessage(content="Tell me about Acne How we prevent from this Problem?")]

#passing this state to workflow.
response = workflow.invoke({'messages':messages},config={"run_name": "AgenticRAGWorkflow"})
response

user_query_or_respond
[2025-09-23 16:06:13,492]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2025-09-23 16:06:14,188]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
grade_document executed
[2025-09-23 16:06:19,396]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
relevant document
geenerate answer executing
[2025-09-23 16:06:20,384]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='Tell me about Acne How we prevent from this Problem?', additional_kwargs={}, response_metadata={}, id='18c77599-3a1b-43fb-9af4-7318548602df'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cG6u09C2GB5AkbWjxEIyz25h', 'function': {'arguments': '{"query":"Acne prevention"}', 'name': 'medical_doc_retriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 161, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CIufVZOU8zO9EZQQBCLyknodFKdTL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1bff4a20-3cb4-49f7-8540-f96088b14309-0', tool_calls=[{'name': 'medical_doc_retriever'

In [23]:
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

Tell me about Acne How we prevent from this Problem?
================================== Ai Message ==================================
Tool Calls:
  medical_doc_retriever (call_cG6u09C2GB5AkbWjxEIyz25h)
 Call ID: call_cG6u09C2GB5AkbWjxEIyz25h
  Args:
    query: Acne prevention
================================= Tool Message =================================
Name: medical_doc_retriever

isotretinoin. It can be controlled by proper treatment, ORGANIZATIONS
with improvement taking two or more months. Acne American Academy of Dermatology. 930 N. Meacham Road,
tends to reappear when treatment stops,but spontaneous- P.O. Box 4014,Schaumburg,IL 60168-4014. (847) 330-
ly improves over time. Inflammatory acne may leave 0230. <http://www.aad.org>.
scars that require further treatment.
Mercedes McLaughlin
Prevention
Acne rosacea see Rosacea
There are no sure ways to prevent acne,but the fol-
Acoustic neurinoma see Aco

In [24]:
messages=[HumanMessage(content="What is langgraph how it work?")]

#passing this state to workflow.
response = workflow.invoke({'messages':messages})
for msg in response['messages']:
    msg.pretty_print()

user_query_or_respond
[2025-09-23 16:06:21,587]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2025-09-23 16:06:22,100]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
grade_document executed
[2025-09-23 16:06:25,873]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ir-relevant document
rewrite question executing
[2025-09-23 16:06:26,502]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
user_query_or_respond
[2025-09-23 16:06:27,659]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
================================ Human Message =================================

What is langgraph how it work?
================================== Ai Message ==================================

I can help you find information on Langgraph and how it works. Let me retrieve relevant details about Langgraph for

In [25]:
from IPython.display import Image, display
workflow = graph.compile()
try:
    display(Image(workflow.get_graph(xray=True).draw_mermaid_png()))
except Exception as e:
    raise CustomException(e,sys)


[2025-09-23 16:06:28,806]-ERROR-41-Python Error getting in this file C:\Users\Admin\AppData\Local\Temp\ipykernel_9456\2086259708.py at Line No 4 and Message Of Error is Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


CustomException: Python Error getting in this file C:\Users\Admin\AppData\Local\Temp\ipykernel_9456\2086259708.py at Line No 4 and Message Of Error is Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`